In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [17]:
class Net (nn.Module):
    
    def __init__ (self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
        
    def forward (self, x):
        
        # Conv 1
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        
        # Conv2
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        # Flatten
        x = x.view(-1, self.num_flat_features(x))
        
        # Fc1
        x = F.relu(self.fc1(x))
        
        # Fc2
        x = F.relu(self.fc2(x))
        
        # Fc3
        x = self.fc3(x)
        
        return x
        

In [43]:
net = Net()

In [44]:
input = torch.rand((1, 1, 32, 32))

In [45]:
output = net(input)

In [46]:
target = torch.arange(1, 11)

In [47]:
target = target.view(1, -1)

In [48]:
loss_function = nn.MSELoss()

In [49]:
loss = loss_function(output, target)

In [50]:
loss

tensor(38.1126)

In [37]:
loss.backward()

In [38]:
print(loss.grad_fn)

In [51]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None


In [52]:
loss.backward()

In [53]:
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad after backward
tensor([ 0.0028, -0.0106, -0.0435, -0.1075,  0.0382, -0.1271])


In [54]:
lr = 0.01

for f in net.parameters():
    f.data.sub_(f.grad.data * lr)

In [56]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [58]:
# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = loss_function(output, target)
loss.backward()
optimizer.step()    # Does the update